# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

#### installed gmaps
- pip install gmaps

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create DataFrame from WeatherPy csv 
weather_info = pd.read_csv("../output_data/city_weather.csv")
weather_info


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bluff,-46.60,168.33,39.00,83,88,3.00,NZ,1595528386
1,Akdepe,42.06,59.38,78.80,57,27,6.93,TM,1595528386
2,San Quintín,30.48,-115.95,69.48,63,47,12.59,MX,1595528386
3,Upernavik,72.79,-56.15,46.33,74,82,11.45,GL,1595528386
4,Punta Arenas,-53.15,-70.92,39.20,80,40,13.87,CL,1595528327
...,...,...,...,...,...,...,...,...,...
551,Dakar,14.69,-17.44,86.00,79,40,9.17,SN,1595528455
552,Marang,5.21,103.21,78.80,88,40,2.24,MY,1595528455
553,Grand Gaube,-20.01,57.66,69.01,60,57,8.99,MU,1595528455
554,Banda Aceh,5.56,95.32,78.57,76,93,3.98,ID,1595528455


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_info[["Lat","Lng"]]

# Store humidity as the weight in hweight
hweight = weather_info["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(map_type="SATELLITE")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=hweight,dissipating=False, max_intensity=100,point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Set perfect weather conditions
hotel_df = weather_info.loc[(weather_info["Max Temp"]>= 65)&(weather_info["Max Temp"]<=70)&(weather_info["Wind Speed"]<10)&(weather_info["Cloudiness"]==0)]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,East London,-33.02,27.91,65.80,52,0,9.37,ZA,1595528311
74,Lagoa,39.05,-27.98,69.85,69,0,2.73,PT,1595528396
163,Lüderitz,-26.65,15.16,69.80,46,0,2.24,NaN,1595528409
182,Ayagoz,47.97,80.44,65.55,38,0,6.98,KZ,1595528412
223,Anapskaya,44.89,37.38,68.00,49,0,8.95,RU,1595528417
292,Malanje,-9.54,16.34,69.75,24,0,0.36,AO,1595528426
310,Karratha,-20.74,116.85,68.56,54,0,4.54,AU,1595528429
503,Mossamedes,-15.20,12.15,69.64,68,0,4.85,AO,1595528450
543,Krasnoarmeysk,51.02,45.70,70.00,29,0,5.01,RU,1595528454


In [8]:
# Drop the other rows that do not fit the criteria above 
# Reset the index and drop previous index number
hotel_df = hotel_df.dropna(how='any')
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,East London,-33.02,27.91,65.80,52,0,9.37,ZA,1595528311
1,Lagoa,39.05,-27.98,69.85,69,0,2.73,PT,1595528396
2,Ayagoz,47.97,80.44,65.55,38,0,6.98,KZ,1595528412
3,Anapskaya,44.89,37.38,68.00,49,0,8.95,RU,1595528417
4,Malanje,-9.54,16.34,69.75,24,0,0.36,AO,1595528426
5,Karratha,-20.74,116.85,68.56,54,0,4.54,AU,1595528429
6,Mossamedes,-15.20,12.15,69.64,68,0,4.85,AO,1595528450
7,Krasnoarmeysk,51.02,45.70,70.00,29,0,5.01,RU,1595528454


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Add "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"]=""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,East London,-33.02,27.91,65.80,52,0,9.37,ZA,1595528311,
1,Lagoa,39.05,-27.98,69.85,69,0,2.73,PT,1595528396,
2,Ayagoz,47.97,80.44,65.55,38,0,6.98,KZ,1595528412,
3,Anapskaya,44.89,37.38,68.00,49,0,8.95,RU,1595528417,
4,Malanje,-9.54,16.34,69.75,24,0,0.36,AO,1595528426,
5,Karratha,-20.74,116.85,68.56,54,0,4.54,AU,1595528429,
6,Mossamedes,-15.20,12.15,69.64,68,0,4.85,AO,1595528450,
7,Krasnoarmeysk,51.02,45.70,70.00,29,0,5.01,RU,1595528454,


In [11]:
# Params dictionary to update each iteration
params = {
            
            "radius":5000,
            "types":"lodging",
            "key":g_key
        
         }
# Use Lat and Long to identify the hotel names
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Base URL to use for request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make the request
    hotel_name = requests.get(base_url, params=params).json()
    
     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][3]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")

In [12]:
# Export to csv file 
# Display dataframe with new Hotel Name information
hotel_df.to_csv("../output_data/hotel_df.csv")
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,East London,-33.02,27.91,65.80,52,0,9.37,ZA,1595528311,Premier Hotels & Resorts
1,Lagoa,39.05,-27.98,69.85,69,0,2.73,PT,1595528396,Quinta do Fragoso
2,Ayagoz,47.97,80.44,65.55,38,0,6.98,KZ,1595528412,Aygerim Hotel
3,Anapskaya,44.89,37.38,68.00,49,0,8.95,RU,1595528417,Irina guest house
4,Malanje,-9.54,16.34,69.75,24,0,0.36,AO,1595528426,Prédio de Agricultura
5,Karratha,-20.74,116.85,68.56,54,0,4.54,AU,1595528429,"Discovery Parks - Pilbara, Karratha"
6,Mossamedes,-15.20,12.15,69.64,68,0,4.85,AO,1595528450,Hotel Katus
7,Krasnoarmeysk,51.02,45.70,70.00,29,0,5.01,RU,1595528454,Aleksandr Domnin


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
# info_box_content displays information from hotel_info when clicked on
markers = gmaps.marker_layer(locations, info_box_content = hotel_info )
fig.add_layer(markers)

# Display figure
fig

# info_box_content source: API documentation¶. (n.d.). Retrieved July 23, 2020, from https://jupyter-gmaps.readthedocs.io/en/latest/api.html

Figure(layout=FigureLayout(height='420px'))